# Atomically Trading Bitcoins Between Forks

Implementation of http://homepages.cs.ncl.ac.uk/patrick.mc-corry/atomically-trading-roger.pdf

In [ ]:
%load_ext autoreload
%autoreload 2

## Init blockchain

In [ ]:
from rpc import Proxy
import bitcoin

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
rpc = Proxy()

In [ ]:
_ = rpc.generate(101)

## Init Alice and Bob

In [ ]:
import random
import hashlib
from collections import namedtuple
import bitcoin.core as bc
import bitcoin.core.script as bs
import bitcoin.wallet as bw

In [ ]:
class User(object):
    def __init__(self, rpc):
        self.rpc = rpc
        self._secrets = []
        self.addresses = []
        self._password = str(hash(self) + random.uniform(0,1)).encode()
    
    def generate_addresses(self, n):
        for _ in range(n):
            addr = self.rpc.getnewaddress()
            secr = self.rpc.dumpprivkey(addr)
            self.addresses.append(addr)
            self._secrets.append(secr)
            
    def pub_key(self, key_id):
        return self._secrets[key_id].pub
    
    @property
    def utxos(self):
        txs = self.rpc.listunspent(addrs=self.addresses)
        return txs
    
    def utxo(self, amount):
        # find the right utxo given `amount`
        utxos = self.utxos
        amounts = [u['amount'] for u in utxos]
        utxo_id = amounts.index(amount)
        utxo = utxos[utxo_id]
        return utxo
    
    def sign_hash(self, key_id, sig_hash, sig_hash_type=bs.SIGHASH_ALL):
        assert key_id >= 0 and key_id < len(self._secrets)
        assert isinstance(sig_hash, bytes)

        secret = self._secrets[key_id]
        sig = secret.sign(sig_hash) + bytes([sig_hash_type])
        return sig
    
    def sign_utxo(self, utxo, tx, vin_id):
        # copy `tx`
        tx = bc.CMutableTransaction.from_tx(tx)
        # get required input
        txin = tx.vin[vin_id]
        assert txin.prevout == utxo['outpoint']
        
        addr = utxo['address']
        key_id = self.addresses.index(addr)
        secret = self._secrets[key_id]
        
        sighash = bs.SignatureHash(addr.to_scriptPubKey(),
                                   tx,
                                   vin_id,
                                   bs.SIGHASH_ALL)
        
        sig = secret.sign(sighash) + bytes([bs.SIGHASH_ALL])
        txin.scriptSig = bs.CScript([sig, secret.pub])
        return tx
    
    def hashed_password(self):
        return bc.Hash160(self._password)
    
    def reveal_password(self):
        return self._password

In [ ]:
ali = User(rpc)
bob = User(rpc)

ali.generate_addresses(6)
bob.generate_addresses(6)

In [ ]:
SWAP_AMOUT = 10 * bc.COIN
CANCEL_AMOUNT = 2

In [ ]:
_ = rpc.sendtoaddress(ali.addresses[0], SWAP_AMOUT)
_ = rpc.sendtoaddress(bob.addresses[0], SWAP_AMOUT)

In [ ]:
_ = rpc.generate(1)

## Create 'timers'

In [ ]:
curr_block = rpc.getinfo()['blocks']
curr_block

In [ ]:
Timers = namedtuple('SwapTimers', ['cancel', 'fork', 'bob', 'ali'])

timers = Timers(cancel=curr_block + 4,
                fork=curr_block + 6,
                bob=curr_block + 8,
                ali=curr_block + 10)
timers

## Define basic scripts

In [ ]:
def get_refund_script(pk, delta):
    assert isinstance(pk, bc.key.CPubKey)
    assert isinstance(delta, int)
    assert delta > 0

    script = bs.CScript([
        delta,
        bs.OP_CHECKLOCKTIMEVERIFY,
        bs.OP_DROP,
        pk,
        bs.OP_CHECKSIG
    ])
    return script

def get_transfer_script(pk1, pk2, h):
    assert isinstance(pk1, bc.key.CPubKey)
    assert isinstance(pk2, bc.key.CPubKey)
    assert isinstance(h, bytes)

    script = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIGVERIFY,
        bs.OP_HASH160,
        h,
        bs.OP_EQUAL
    ])

    return script

def get_cancel_script(pk1, pk2):
    assert isinstance(pk1, bc.key.CPubKey)
    assert isinstance(pk2, bc.key.CPubKey)

    script = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

    return script

In [ ]:
def get_ali_deposit_script(ali, bob, timers):
    # TODO: implement forfeit condition
    refund_script = get_refund_script(ali.pub_key(1), timers.ali)
    cancel_script = get_cancel_script(ali.pub_key(3), bob.pub_key(3))

    h = ali.hashed_password()
    transfer_script = get_transfer_script(ali.pub_key(2), bob.pub_key(2), h)

    deposit = bs.CScript([
        bs.OP_IF,
            bs.OP_IF,
                *list(refund_script),
            bs.OP_ELSE,
                *list(transfer_script),
            bs.OP_ENDIF,
        bs.OP_ELSE,
            *list(cancel_script),
        bs.OP_ENDIF])
    return deposit

def get_bob_deposit_script(ali, bob, timers):
    # TODO: implement forfeit condition
    refund_script = get_refund_script(bob.pub_key(1), timers.bob)
    cancel_script = get_cancel_script(ali.pub_key(3), bob.pub_key(3))

    h = ali.hashed_password()
    transfer_script = get_transfer_script(ali.pub_key(2), bob.pub_key(2), h)

    deposit = bs.CScript([
        bs.OP_IF,
            bs.OP_IF,
                *list(refund_script),
            bs.OP_ELSE,
                *list(transfer_script),
            bs.OP_ENDIF,
        bs.OP_ELSE,
            *list(cancel_script),
        bs.OP_ENDIF])
    return deposit

## Define transactions

In [ ]:
def get_fund_tx(ali,
                bob,
                ali_utxo,
                bob_utxo,
                timers,
                cancel_amount=CANCEL_AMOUNT,
                fee=0):
    assert cancel_amount > 0
    assert fee >= 0

    ali_txin = bc.CMutableTxIn(ali_utxo['outpoint'], nSequence=0)
    bob_txin = bc.CMutableTxIn(bob_utxo['outpoint'], nSequence=0)

    ali_deposit = get_ali_deposit_script(ali, bob, timers)
    bob_deposit = get_bob_deposit_script(ali, bob, timers)

    ali_deposit_txout = bc.CMutableTxOut(
        ali_utxo['amount'] - cancel_amount/2 - fee/2,
        ali_deposit)
    bob_deposit_txout = bc.CMutableTxOut(
        bob_utxo['amount'] - cancel_amount/2 - fee/2,
        bob_deposit)

    # TODO: implement commit condition
    cancel_script = get_cancel_script(ali.pub_key(2), bob.pub_key(2))
    cancel_txout = bc.CMutableTxOut(cancel_amount, cancel_script)

    vins = [ali_txin, bob_txin]
    vouts = [ali_deposit_txout,
             bob_deposit_txout, 
             cancel_txout]
    fund_tx = bc.CMutableTransaction(vins, vouts)

    fund_tx = ali.sign_utxo(ali_utxo, fund_tx, 0)
    fund_tx = bob.sign_utxo(bob_utxo, fund_tx, 1)

    return fund_tx

In [ ]:
def get_cancel_tx(fund_tx, ali, bob, fee=0):
    fund_tx_id = fund_tx.GetTxid()
    ali_cancel_txin = bc.CMutableTxIn(bc.COutPoint(fund_tx_id, 0),
                                      nSequence=0)
    bob_cancel_txin = bc.CMutableTxIn(bc.COutPoint(fund_tx_id, 1),
                                      nSequence=0)
    can_cancel_txin = bc.CMutableTxIn(bc.COutPoint(fund_tx_id, 2),
                                      nSequence=0)

    cancel_amount = fund_tx.vout[2].nValue
    assert CANCEL_AMOUNT == cancel_amount

    ali_cancel_txout = bc.CMutableTxOut(
        fund_tx.vout[0].nValue + cancel_amount/2 - fee/2,
        ali.addresses[5].to_scriptPubKey())
    bob_cancel_txout = bc.CMutableTxOut(
        fund_tx.vout[1].nValue + cancel_amount/2 - fee/2,
        bob.addresses[5].to_scriptPubKey())

    vins = [ali_cancel_txin, bob_cancel_txin, can_cancel_txin]
    vouts = [ali_cancel_txout, bob_cancel_txout]
    cancel_tx = bc.CMutableTransaction(vins, vouts)

    ali_cancel_sighash = bs.SignatureHash(
        fund_tx.vout[0].scriptPubKey,
        cancel_tx,
        0,
        bs.SIGHASH_ALL)

    bob_cancel_sighash = bs.SignatureHash(
        fund_tx.vout[1].scriptPubKey,
        cancel_tx,
        1,
        bs.SIGHASH_ALL)

    can_cancel_sighash = bs.SignatureHash(
        fund_tx.vout[2].scriptPubKey,
        cancel_tx,
        2,
        bs.SIGHASH_ALL)

    ali_cancel_ali_sig = ali.sign_hash(3, ali_cancel_sighash)
    ali_cancel_bob_sig = bob.sign_hash(3, ali_cancel_sighash)

    bob_cancel_ali_sig = ali.sign_hash(3, bob_cancel_sighash)
    bob_cancel_bob_sig = bob.sign_hash(3, bob_cancel_sighash)

    can_cancel_ali_sig = ali.sign_hash(2, can_cancel_sighash)
    can_cancel_bob_sig = bob.sign_hash(2, can_cancel_sighash)

    ali_cancel_txin.scriptSig = bs.CScript(
        [bs.OP_0, ali_cancel_ali_sig, ali_cancel_bob_sig, bs.OP_0])

    bob_cancel_txin.scriptSig = bs.CScript(
        [bs.OP_0, bob_cancel_ali_sig, bob_cancel_bob_sig, bs.OP_0])

    can_cancel_txin.scriptSig = bs.CScript(
        [bs.OP_0, can_cancel_ali_sig, can_cancel_bob_sig])

    return cancel_tx

In [ ]:
def get_ali_transfer_tx(fund_tx, ali, bob, fee=0):
    fund_tx_id = fund_tx.GetTxid()
    ali_txin = bc.CMutableTxIn(bc.COutPoint(fund_tx_id, 0),
                               nSequence=0)
    bob_txin = bc.CMutableTxIn(bc.COutPoint(fund_tx_id, 1),
                               nSequence=0)

    amount = fund_tx.vout[0].nValue + fund_tx.vout[1].nValue

    ali_txout = bc.CMutableTxOut(
        amount - fee,
        ali.addresses[5].to_scriptPubKey())

    vins = [ali_txin, bob_txin]
    vouts = [ali_txout]
    ali_tx = bc.CMutableTransaction(vins, vouts)

    ali_deposit_sighash = bs.SignatureHash(
        fund_tx.vout[0].scriptPubKey,
        ali_tx,
        0,
        bs.SIGHASH_ALL)

    bob_deposit_sighash = bs.SignatureHash(
        fund_tx.vout[1].scriptPubKey,
        ali_tx,
        1,
        bs.SIGHASH_ALL)

    ali_deposit_ali_sig = ali.sign_hash(2, ali_deposit_sighash)
    ali_deposit_bob_sig = bob.sign_hash(2, ali_deposit_sighash)

    bob_deposit_ali_sig = ali.sign_hash(2, bob_deposit_sighash)
    bob_deposit_bob_sig = bob.sign_hash(2, bob_deposit_sighash)

    ali_txin.scriptSig = bs.CScript(
        [bs.OP_0, ali_deposit_ali_sig, ali_deposit_bob_sig, bs.OP_0, bs.OP_1])

    bob_txin.scriptSig = bs.CScript(
        [bs.OP_0, bob_deposit_ali_sig, bob_deposit_bob_sig, bs.OP_0, bs.OP_1])

    return ali_tx

# TODO: implement ali -> bob transfer tx

## Create $T^{Fund}$ transaction

In [ ]:
ali_utxo = ali.utxo(SWAP_AMOUT)
bob_utxo = bob.utxo(SWAP_AMOUT)

In [ ]:
fund_tx = get_fund_tx(ali, bob, ali_utxo, bob_utxo, timers)

In [ ]:
# https://github.com/petertodd/python-bitcoinlib/blob/5e150ac4a50791e6293752ceef8647b9bb3273c0/examples/timestamp-op-ret.py#L66
FEE_PER_BYTE = 0.00025 * bc.COIN/1000

In [ ]:
n_bytes = len(fund_tx.serialize())
fee = n_bytes * FEE_PER_BYTE
n_bytes, fee

In [ ]:
fund_tx = get_fund_tx(ali, bob, ali_utxo, bob_utxo, timers, fee=fee)

In [ ]:
fund_tx_id = rpc.sendrawtransaction(fund_tx)

In [ ]:
_ = rpc.generate(1)

In [ ]:
assert rpc.gettransaction(fund_tx_id)['confirmations'] == 1

## Create $T^{cancel}$

In [ ]:
fund_tx = rpc.getrawtransaction(fund_tx_id)

In [ ]:
cancel_tx = get_cancel_tx(fund_tx, ali, bob)

In [ ]:
n_bytes = len(cancel_tx.serialize())
fee = n_bytes * FEE_PER_BYTE
n_bytes, fee

In [ ]:
cancel_tx = get_cancel_tx(fund_tx, ali, bob, fee=fee)

In [ ]:
# spend T_cancel
# cancel_tx_id = rpc.sendrawtransaction(cancel_tx)
# _ = rpc.generate(1)
# assert rpc.gettransaction(cancel_tx_id)['confirmations'] == 1

## Create $T^{transfer}$

In [ ]:
fund_tx = rpc.getrawtransaction(fund_tx_id)

In [ ]:
transfer_tx = get_ali_transfer_tx(fund_tx, ali, bob)

# Reveal Ali's password
transfer_tx.vin[0].scriptSig = bc.CScript([ali.reveal_password()] + list(transfer_tx.vin[0].scriptSig))
transfer_tx.vin[1].scriptSig = bc.CScript([ali.reveal_password()] + list(transfer_tx.vin[1].scriptSig))

In [ ]:
n_bytes = len(transfer_tx.serialize())
fee = n_bytes * FEE_PER_BYTE
n_bytes, fee

In [ ]:
transfer_tx = get_ali_transfer_tx(fund_tx, ali, bob, fee=fee)

# Reveal Ali's password
transfer_tx.vin[0].scriptSig = bc.CScript([ali.reveal_password()] + list(transfer_tx.vin[0].scriptSig))
transfer_tx.vin[1].scriptSig = bc.CScript([ali.reveal_password()] + list(transfer_tx.vin[1].scriptSig))

In [ ]:
transfer_tx_id = rpc.sendrawtransaction(transfer_tx)
_ = rpc.generate(1)
assert rpc.gettransaction(transfer_tx_id)['confirmations'] == 1

## Create $T^{commit}$

In [ ]:
# TODO: Implement commit tx